In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [15]:
#unsharp masking: enhances edges, making them more defined
img=cv2.imread("im2.jpg", cv2.IMREAD_GRAYSCALE)
blurred = cv2.GaussianBlur(img, (0, 0), 5)
unsharp_mask = cv2.subtract(img, blurred)
img= cv2.add(img, unsharp_mask)
cv2.imshow("original image:",img)
cv2.waitKey(0)
h, w = img.shape
imnew=np.zeros((h, w), dtype=np.uint8)
img1=cv2.medianBlur(img,9)
l=[]

for x in range(h-1):
    for y in range(w-1):
        if(img1[x,y]>40):#only listing the points which are part of the eye image and not the black background
            l.append(img[x,y])
            imnew[x,y]=100
l=np.array(l)
cv2.imshow("",imnew)
cv2.waitKey(0)

#finding the mean and standard deviation of the points of the eye to get an idea of the intesity values
st=np.std(l)
me=np.average(l)
#initialising separate cup and disc images for binarization
img_disc = np.zeros((h, w), dtype=np.uint8)
img_cup = np.zeros((h, w), dtype=np.uint8)
#binary thresholding with threshold as mean+2.2*stddeviation for disc extraction
for x in range(h-1):
    for y in range(w-1):
        if(img[x,y]>me+2.2*st):
            img_disc[x,y]=150
        else:
            img_disc[x,y]=0
#reducing false positives for disc. eliminating small positive regions
for x in range(h-1):
    for y in range(w-1):
        v=30
        xl=max(x-v,0)
        xh=min(x+v,h-1)
        yl=max(y-v,0)
        yh=min(y+v,w-1)
        if(img_disc[xl,y]!=150 and img_disc[xh,y]!=150 and img_disc[x,yl]!=150 and img_disc[x,yh]!=150):
            img_disc[x,y]=0


print("height",h,w)
img_disc=cv2.medianBlur(img_disc,19)#median blurring to further reduce relatively smaller false positive regions
#binary thresholding with threshold as mean+3.4*stddeviation for cup extraction(cup is brighter than disc)
for x in range(h-1):
    for y in range(w-1):
        if(img[x,y]>me+3.6*st and img_disc[x,y]==150):#extra condition to make sure cup lies in disc
            img_cup[x,y]=230
        else:
            img_cup[x,y]=0
cv2.imshow("disc",img_disc)
cv2.waitKey(0)
#reducing impact of low-intensity nerve regions on cup area, making it a more uniform and solid region
for x in range(h-1):
    for y in range(w-1):
        xl=max(x-15,0)
        xh=min(x+15,h-1)
        yl=max(y-15,0)
        yh=min(y+15,w-1)
        if((img_cup[xl,y]==230 and img_cup[xh,y]==230) or (img_cup[x,yl]==230 and img_cup[x,yh]==230) and img_disc[x,y]==150):
            img_cup[x,y]=230
#reducing impact of low-intensity nerve regions on cup area, making it a more uniform and solid region
for x in range(h-1):
    for y in range(w-1):
        xl=max(x-15,0)
        xh=min(x+15,h-1)
        yl=max(y-15,0)
        yh=min(y+15,w-1)
        if((img_cup[xl,y]==230 and img_cup[xh,y]==230) or (img_cup[x,yl]==230 and img_cup[x,yh]==230) and img_disc[x,y]==150):
            img_cup[x,y]=230
img_cup=cv2.medianBlur(img_cup,19)#median blurring to get more solid cup region figure
cv2.imshow("cup",img_cup)
cv2.waitKey(0)

print("standard deviation:",st)
imgfinal = np.zeros((h, w), dtype=np.uint8)  # Use uint8 for intermediate calculations
intensities=[]
counts={}
#merging all 3 of the images: eye region,disc region and cup region
#also determining the number of pixels in the cup and disc region for CDR determination
d=0
c=0
for x in range(h-1):
    for y in range(w-1):
        imgfinal[x, y] = (int(imnew[x, y]) + int(img_disc[x, y])+int(img_cup[x,y])) // 3  # Use // for integer division
        if(imgfinal[x,y]!=0):
            intensities.append(imgfinal[x,y])
        if(img_disc[x,y]>30):
            d+=1
        if(img_cup[x,y]>60):
            c+=1

        
intensities=np.unique(intensities)
intensities.sort()
ctdr=c/d#calcuation of the cup to disc ratio

print("intensities",intensities)
#print("counts",counts)
print("cup to disc ratio:",ctdr)
imgfinal=cv2.medianBlur(imgfinal,19)
cv2.imshow("Final image", imgfinal)#displaying final image containing all of the regions

cv2.waitKey(0)
cv2.destroyAllWindows()

height 576 768
standard deviation: 22.52723406316484
intensities [ 33  83 160]
cup to disc ratio: 0.15611814345991562
